# Raw Data Distribution Analysis

In [1]:
from main_head import *

# Parameter Fixed

In [2]:
# epsilon is the fixed parameter regarding optimizing CVaR, representing 1-\eta
epsilon = 0.05

# for the whole rolling approach, the number of days we consider in each period
## Here, days represent the unit in the multi-stage CVaR Portfolio Optimization Problem, the value represent the period where we fix the alocation policy.
## It is not just the physical days. Instead, in our paper, we consider the month as a unit. 
rolling_day = 1

# It denotes whether we include the shortsale constraints in our optimization model
## 0 represents that we incorporate the shortsale constraint: x_i >= 0 in our paper
shortsale_sign = 0

# cross validation choice
## cv_type == -1 means no  
## cv_type == 0 means gridsearch
## cv_type == 1 means pre_cv_lp-based search
## cv_type == 2 means using theta from '../factor model/fcvar_hmm_theta_industryindices.csv'
cv_type = 1

# the K-fold number in cross validation
fold_number = 4

# the possible theta values used in the cross validation
theta_list = [0.025, 0.05, 0.075, 0.1, 0.125]

# whether we take the risk-free rate into account
## true means we need to minus the risk-free rate from the dataset and false means not
## Because DeMiguel's paper does not incorporate risk free rate, we do not incorporate that item too in the test statistics computation.
sharpe_ratio_open = False

#to make it more clearly, unit = 1 means the unit of portfolio dta is x%, unit = 100 means the unit of portfolio dta is x. 
unit = 1

# Load the Data

In [5]:
def MD_analysis(port_num, data_type, cal_type = 'dif', t_start = 19970101, t_split = 20061231, t_end = 201904):
    #choose the dataset name and the file path
    portfolio_number = port_num
    freq = "Monthly"
    #Here as we all use the Monthly Data Set, we can fix that parameter

    value = ""
    #eq/not eq csv in the . Here we all use the original dataset class.

    #dta type
    ## In DeMiguel's paper, the dta type are selected to be one in ['MKT', 'Country', 'S&P_sectors', 'IndustryPortfolios', 'FF']
    ## When his dataset extended to 2019 Q1, the dta type are selected to be one in ['MKT2', 'Country2', 'S&P_sectors2', 'IndustryPortfolios2', 'FF22']
    ## Their corresponding portfolio number is 3, 9, 11, 11, 21(24)
    ## And in our case study, dta type are selected to be 'IndustryIndices' (16)
    data_type = data_type

    #select part of the data in the .csv file.
    ## from start_time to train_test_split as the default train dataset; from train_test_split (not include) as the test dataset.
    start_time = t_start
    end_time = t_end
    train_test_split = t_split

    #data input
    Input_csv = Input(portfolio_number, freq, value, start_time, end_time, train_test_split, data_type)
    [data_head, data_parameter, csv_name] = Input_csv.parameter_output()
    [df_select, df_train] = Input_csv.data_load()

    df_factor = Input_csv.three_factor_load()
    #we do not include the risk free rate item into the computation of sharpe ratio
    if sharpe_ratio_open == False:
        rfr_data = 0
    else:
        rfr_data = Input_csv.risk_free_rate()


    #rolling window method
    num_of_sample = len(df_select)
    num_of_train = len(df_train)
    i = 0
    MD_series = np.zeros(num_of_sample - num_of_train)
    while i < num_of_sample - num_of_train:   
        train_return = np.array(df_select[i: (i + num_of_train)])
        if cal_type == 'dif':
            MD = np.linalg.norm(np.max(train_return, axis = 0) - np.min(train_return, axis = 0), ord = 1)
        elif cal_type == 'minimax':
            max_array = np.max(train_return, axis = 0)
            min_array = np.max(train_return, axis = 0)
            MD = np.linalg.norm(np.array([max(abs(a_e), abs(b_e)) for (a_e, b_e) in zip(max_array, min_array)]), ord = 1)
        elif cal_type == 'abs_minimax':
            MD = np.max(train_return) - np.min(train_return)
        i += 1
        print(i, MD)
        MD_series[i-1] = MD
    print(np.max(MD_series), np.min(MD_series))

In [7]:
MD_analysis(11, 'IndustryPortfolios2', 'abs_minimax')

1 0.4732
2 0.4732
3 0.4732
4 0.4732
5 0.4732
6 0.4732
7 0.4732
8 0.4732
9 0.4732
10 0.4732
11 0.4732
12 0.4732
13 0.4732
14 0.4732
15 0.4732
16 0.4732
17 0.4732
18 0.4732
19 0.4732
20 0.4732
21 0.4732
22 0.4732
23 0.5399
24 0.5399
25 0.5399
26 0.5399
27 0.5399
28 0.5399
29 0.7525999999999999
30 0.7525999999999999
31 0.7525999999999999
32 0.7525999999999999
33 0.7525999999999999
34 0.7525999999999999
35 0.7525999999999999
36 0.7525999999999999
37 0.7525999999999999
38 0.7525999999999999
39 0.7525999999999999
40 0.7525999999999999
41 0.7525999999999999
42 0.7525999999999999
43 0.7525999999999999
44 0.7525999999999999
45 0.7525999999999999
46 0.7525999999999999
47 0.7525999999999999
48 0.7525999999999999
49 0.7525999999999999
50 0.7525999999999999
51 0.7525999999999999
52 0.7525999999999999
53 0.7525999999999999
54 0.7525999999999999
55 0.7525999999999999
56 0.7525999999999999
57 0.7525999999999999
58 0.7525999999999999
59 0.7525999999999999
60 0.7525999999999999
61 0.7525999999999999
62 

In [4]:
MD_analysis(11, 'IndustryPortfolios2', 'minimax')

1 1.5798
2 1.5798
3 1.5798
4 1.5798
5 1.5798
6 1.5798
7 1.5798
8 1.5798
9 1.5798
10 1.5798
11 1.5798
12 1.5798
13 1.5798
14 1.5798
15 1.5798
16 1.5798
17 1.5798
18 1.5798
19 1.5798
20 1.5798
21 1.5798
22 1.5749000000000002
23 1.5451
24 1.5451
25 1.5451
26 1.5451
27 1.5451
28 1.5523999999999998
29 1.9357
30 1.9357
31 1.9357
32 1.9357
33 1.9357
34 1.9357
35 1.9357
36 1.9357
37 1.9222
38 1.9222
39 1.9222
40 1.8907999999999998
41 1.8907999999999998
42 1.8907999999999998
43 1.8578999999999999
44 1.8578999999999999
45 1.8417999999999999
46 1.8691
47 1.8691
48 1.8691
49 1.8691
50 1.8691
51 1.8691
52 1.8691
53 1.8691
54 1.8691
55 1.8691
56 1.8691
57 1.8691
58 1.8691
59 1.8806999999999998
60 1.8806999999999998
61 1.8806999999999998
62 1.8806999999999998
63 1.8806999999999998
64 1.8806999999999998
65 1.8806999999999998
66 1.8806999999999998
67 1.8806999999999998
68 1.8806999999999998
69 1.8806999999999998
70 1.8806999999999998
71 1.7519999999999998
72 1.7087999999999999
73 1.7087999999999999
74 

In [8]:
MD_analysis(11, 'S&P_sectors2', 'abs_minimax')

1 0.6930000000000001
2 0.6930000000000001
3 0.6930000000000001
4 0.6930000000000001
5 0.6930000000000001
6 0.6930000000000001
7 0.6930000000000001
8 0.6930000000000001
9 0.6930000000000001
10 0.6930000000000001
11 0.6930000000000001
12 0.6930000000000001
13 0.6930000000000001
14 0.6930000000000001
15 0.6930000000000001
16 0.6930000000000001
17 0.6930000000000001
18 0.6930000000000001
19 0.6930000000000001
20 0.6930000000000001
21 0.6930000000000001
22 0.6930000000000001
23 0.6930000000000001
24 0.6930000000000001
25 0.6930000000000001
26 0.6930000000000001
27 0.6930000000000001
28 0.6930000000000001
29 0.6930000000000001
30 0.6930000000000001
31 0.6930000000000001
32 0.6930000000000001
33 0.6930000000000001
34 0.6930000000000001
35 0.6930000000000001
36 0.6930000000000001
37 0.6930000000000001
38 0.6930000000000001
39 0.6930000000000001
40 0.6930000000000001
41 0.6930000000000001
42 0.6930000000000001
43 0.6930000000000001
44 0.6930000000000001
45 0.6930000000000001
46 0.69300000000000

In [9]:
MD_analysis(3, 'MKT2', 'abs_minimax')

1 0.3801
2 0.3801
3 0.3801
4 0.3801
5 0.3801
6 0.3801
7 0.3801
8 0.3801
9 0.3801
10 0.3801
11 0.3801
12 0.3801
13 0.3801
14 0.3801
15 0.3801
16 0.3801
17 0.3801
18 0.3801
19 0.3801
20 0.3801
21 0.3801
22 0.3801
23 0.3842
24 0.3842
25 0.3842
26 0.3842
27 0.3842
28 0.3842
29 0.3842
30 0.3842
31 0.3842
32 0.3842
33 0.3842
34 0.3842
35 0.3842
36 0.3842
37 0.3842
38 0.3842
39 0.3003
40 0.3003
41 0.3003
42 0.3003
43 0.29810000000000003
44 0.29810000000000003
45 0.29810000000000003
46 0.29810000000000003
47 0.29810000000000003
48 0.29810000000000003
49 0.29810000000000003
50 0.29810000000000003
51 0.2741
52 0.2741
53 0.2741
54 0.2741
55 0.2741
56 0.2741
57 0.2741
58 0.2741
59 0.2858
60 0.2858
61 0.2858
62 0.2858
63 0.2858
64 0.2858
65 0.2858
66 0.2858
67 0.2858
68 0.2858
69 0.2858
70 0.2858
71 0.2858
72 0.2858
73 0.2858
74 0.2858
75 0.2858
76 0.2858
77 0.2858
78 0.2858
79 0.2858
80 0.2858
81 0.2858
82 0.2858
83 0.2858
84 0.2858
85 0.2858
86 0.2858
87 0.2858
88 0.2858
89 0.2858
90 0.2858
91 0.

In [10]:
MD_analysis(9, 'Country2', 'abs_minimax')

1 0.46740219899999996
2 0.46740219899999996
3 0.46740219899999996
4 0.46740219899999996
5 0.46740219899999996
6 0.46740219899999996
7 0.46740219899999996
8 0.46740219899999996
9 0.46740219899999996
10 0.46740219899999996
11 0.46740219899999996
12 0.46740219899999996
13 0.46740219899999996
14 0.46740219899999996
15 0.46740219899999996
16 0.46740219899999996
17 0.46740219899999996
18 0.46740219899999996
19 0.46740219899999996
20 0.46740219899999996
21 0.46740219899999996
22 0.46740219899999996
23 0.495438113
24 0.495438113
25 0.495438113
26 0.495438113
27 0.495438113
28 0.495438113
29 0.495438113
30 0.495438113
31 0.495438113
32 0.495438113
33 0.495438113
34 0.495438113
35 0.495438113
36 0.495438113
37 0.495438113
38 0.495438113
39 0.495438113
40 0.495438113
41 0.495438113
42 0.495438113
43 0.495438113
44 0.495438113
45 0.495438113
46 0.495438113
47 0.495438113
48 0.495438113
49 0.495438113
50 0.495438113
51 0.495438113
52 0.495438113
53 0.495438113
54 0.495438113
55 0.495438113
56 0.495

In [12]:
MD_analysis(21, 'FF2', 'abs_minimax')

1 0.664107
2 0.664107
3 0.664107
4 0.664107
5 0.664107
6 0.664107
7 0.664107
8 0.664107
9 0.664107
10 0.664107
11 0.664107
12 0.664107
13 0.664107
14 0.664107
15 0.664107
16 0.664107
17 0.664107
18 0.664107
19 0.664107
20 0.664107
21 0.628641
22 0.628641
23 0.680678
24 0.680678
25 0.680678
26 0.680678
27 0.680678
28 0.680678
29 0.680678
30 0.680678
31 0.680678
32 0.680678
33 0.680678
34 0.680678
35 0.680678
36 0.680678
37 0.680678
38 0.680678
39 0.540223
40 0.540223
41 0.540223
42 0.540223
43 0.46165900000000004
44 0.46165900000000004
45 0.46165900000000004
46 0.46165900000000004
47 0.46165900000000004
48 0.46165900000000004
49 0.46165900000000004
50 0.46165900000000004
51 0.46165900000000004
52 0.46165900000000004
53 0.46165900000000004
54 0.46165900000000004
55 0.46165900000000004
56 0.46165900000000004
57 0.46165900000000004
58 0.46165900000000004
59 0.46165900000000004
60 0.46165900000000004
61 0.46165900000000004
62 0.46165900000000004
63 0.46165900000000004
64 0.46165900000000004

In [13]:
MD_analysis(24, 'FF2', 'abs_minimax')

1 0.664107
2 0.664107
3 0.664107
4 0.664107
5 0.664107
6 0.664107
7 0.664107
8 0.664107
9 0.664107
10 0.664107
11 0.664107
12 0.664107
13 0.664107
14 0.664107
15 0.664107
16 0.664107
17 0.664107
18 0.664107
19 0.664107
20 0.664107
21 0.661016
22 0.661016
23 0.680678
24 0.680678
25 0.680678
26 0.680678
27 0.680678
28 0.680678
29 0.754416
30 0.754416
31 0.754416
32 0.754416
33 0.754416
34 0.754416
35 0.754416
36 0.754416
37 0.754416
38 0.754416
39 0.613961
40 0.613961
41 0.613961
42 0.613961
43 0.535397
44 0.535397
45 0.535397
46 0.535397
47 0.535397
48 0.535397
49 0.535397
50 0.535397
51 0.535397
52 0.535397
53 0.535397
54 0.535397
55 0.535397
56 0.535397
57 0.535397
58 0.535397
59 0.535397
60 0.535397
61 0.535397
62 0.535397
63 0.535397
64 0.535397
65 0.535397
66 0.535397
67 0.535397
68 0.535397
69 0.535397
70 0.535397
71 0.535397
72 0.535397
73 0.535397
74 0.535397
75 0.535397
76 0.535397
77 0.535397
78 0.535397
79 0.535397
80 0.535397
81 0.535397
82 0.535397
83 0.535397
84 0.535397
8

In [14]:
MD_analysis(16, 'IndustryIndices', 'abs_minimax', 20040101, 20061231, 201905)

1 0.47709999999999997
2 0.47709999999999997
3 0.47709999999999997
4 0.47709999999999997
5 0.3609
6 0.3609
7 0.3609
8 0.32989999999999997
9 0.32989999999999997
10 0.3256
11 0.3256
12 0.3256
13 0.3073
14 0.3073
15 0.3073
16 0.30900000000000005
17 0.30900000000000005
18 0.30900000000000005
19 0.311066242
20 0.311066242
21 0.4145
22 0.4145
23 0.5153
24 0.5153
25 0.5153
26 0.6184000000000001
27 0.6184000000000001
28 0.6184000000000001
29 0.6184000000000001
30 0.6387
31 0.6387
32 0.6387
33 0.6387
34 0.6387
35 0.6387
36 0.6387
37 0.6387
38 0.6387
39 0.6387
40 0.6387
41 0.6387
42 0.6387
43 0.6387
44 0.6387
45 0.6387
46 0.6387
47 0.6387
48 0.6387
49 0.6387
50 0.6387
51 0.6387
52 0.6387
53 0.6387
54 0.6387
55 0.6387
56 0.6387
57 0.6387
58 0.6387
59 0.5841000000000001
60 0.5841000000000001
61 0.5841000000000001
62 0.5841000000000001
63 0.5841000000000001
64 0.5841000000000001
65 0.5841000000000001
66 0.5624
67 0.5624
68 0.5624
69 0.5624
70 0.5624
71 0.5624
72 0.5624
73 0.5624
74 0.5624
75 0.5624


In [5]:
MD_analysis(11, 'IndustryPortfolios2')

1 3.3409000000000004
2 3.3409000000000004
3 3.3409000000000004
4 3.3409000000000004
5 3.3409000000000004
6 3.3409000000000004
7 3.3409000000000004
8 3.3409000000000004
9 3.3409000000000004
10 3.3409000000000004
11 3.3409000000000004
12 3.3409000000000004
13 3.3409000000000004
14 3.3409000000000004
15 3.3409000000000004
16 3.3409000000000004
17 3.3409000000000004
18 3.3409000000000004
19 3.3422
20 3.3796
21 3.1364
22 3.1373
23 3.5526000000000004
24 3.5526000000000004
25 3.5526000000000004
26 3.5526000000000004
27 3.5561000000000003
28 3.5634
29 3.9467000000000003
30 3.9467000000000003
31 3.9467000000000003
32 3.9467000000000003
33 3.9467000000000003
34 3.9467000000000003
35 3.9467000000000003
36 3.9467000000000003
37 3.9332000000000003
38 3.9332000000000003
39 3.9332000000000003
40 3.9018
41 3.9018
42 3.9018
43 3.8689
44 3.8689
45 3.8528
46 3.8801
47 3.8801
48 3.8801
49 3.8801
50 3.8801
51 3.8099
52 3.8099
53 3.8099
54 3.8099
55 3.8099
56 3.8099
57 3.8099
58 3.8028
59 3.8144
60 3.8144
6

In [6]:
MD_analysis(3, 'MKT2')

1 0.8383
2 0.8383
3 0.8383
4 0.8383
5 0.8383
6 0.8383
7 0.8383
8 0.8383
9 0.8383
10 0.8383
11 0.8383
12 0.8383
13 0.8383
14 0.8383
15 0.8383
16 0.8383
17 0.8383
18 0.8383
19 0.8383
20 0.8383
21 0.7847
22 0.7847
23 0.8498000000000001
24 0.8498000000000001
25 0.8498000000000001
26 0.8716
27 0.8716
28 0.8789
29 0.8912
30 0.8912
31 0.8912
32 0.8912
33 0.8912
34 0.8912
35 0.8912
36 0.8912
37 0.8912
38 0.8912
39 0.8073
40 0.7084
41 0.7024
42 0.7016
43 0.6493
44 0.6493
45 0.6493
46 0.6493
47 0.6493
48 0.6493
49 0.6493
50 0.6493
51 0.5985
52 0.5985
53 0.5985
54 0.5985
55 0.5985
56 0.5985
57 0.5985
58 0.5882000000000001
59 0.5854999999999999
60 0.5854999999999999
61 0.5854999999999999
62 0.5854999999999999
63 0.5854999999999999
64 0.5854999999999999
65 0.5854999999999999
66 0.5854999999999999
67 0.5854999999999999
68 0.5751
69 0.5751
70 0.5751
71 0.5751
72 0.5751
73 0.5751
74 0.5751
75 0.5751
76 0.5751
77 0.5751
78 0.5751
79 0.5751
80 0.5751
81 0.5751
82 0.5751
83 0.5751
84 0.5751
85 0.5751
86 

In [7]:
MD_analysis(3, 'MKT', 'dif', 19630701, 19730631, 200412)

1 0.4719
2 0.4719
3 0.4719
4 0.4719
5 0.4719
6 0.49779999999999996
7 0.49779999999999996
8 0.5083
9 0.5083
10 0.5083
11 0.5083
12 0.5083
13 0.5083
14 0.5083
15 0.5083
16 0.5083
17 0.6226
18 0.6226
19 0.6226
20 0.6459
21 0.6459
22 0.6459
23 0.6459
24 0.6459
25 0.6459
26 0.6459
27 0.6459
28 0.6459
29 0.6459
30 0.6459
31 0.6459
32 0.6513
33 0.6513
34 0.6513
35 0.6513
36 0.6513
37 0.6513
38 0.6513
39 0.6513
40 0.6513
41 0.6513
42 0.6513
43 0.6513
44 0.6513
45 0.6513
46 0.6513
47 0.6513
48 0.6513
49 0.6513
50 0.6513
51 0.6513
52 0.6513
53 0.6513
54 0.6513
55 0.6513
56 0.6513
57 0.6513
58 0.6513
59 0.6513
60 0.6513
61 0.6513
62 0.6513
63 0.6513
64 0.6513
65 0.6738000000000001
66 0.6738000000000001
67 0.6738000000000001
68 0.6738000000000001
69 0.6738000000000001
70 0.6738000000000001
71 0.6738000000000001
72 0.6738000000000001
73 0.6738000000000001
74 0.6738000000000001
75 0.6738000000000001
76 0.6738000000000001
77 0.6738000000000001
78 0.6738000000000001
79 0.6738000000000001
80 0.67380000

In [8]:
MD_analysis(3, 'MKT2', 'minimax')

1 0.41990000000000005
2 0.41990000000000005
3 0.41990000000000005
4 0.41990000000000005
5 0.41990000000000005
6 0.41990000000000005
7 0.41990000000000005
8 0.41990000000000005
9 0.41990000000000005
10 0.41990000000000005
11 0.41990000000000005
12 0.41990000000000005
13 0.41990000000000005
14 0.41990000000000005
15 0.41990000000000005
16 0.41990000000000005
17 0.41990000000000005
18 0.41990000000000005
19 0.41990000000000005
20 0.41990000000000005
21 0.41990000000000005
22 0.41990000000000005
23 0.41990000000000005
24 0.41990000000000005
25 0.41990000000000005
26 0.41990000000000005
27 0.41990000000000005
28 0.4272
29 0.4395
30 0.4395
31 0.4395
32 0.4395
33 0.4395
34 0.4395
35 0.4395
36 0.4395
37 0.4395
38 0.4395
39 0.3556
40 0.3556
41 0.3556
42 0.3556
43 0.3033
44 0.3033
45 0.3033
46 0.3033
47 0.3033
48 0.3033
49 0.3033
50 0.3033
51 0.2525
52 0.2525
53 0.2525
54 0.2525
55 0.2525
56 0.2525
57 0.2525
58 0.2525
59 0.24980000000000002
60 0.24980000000000002
61 0.24980000000000002
62 0.2498

In [9]:
MD_analysis(9, 'Country2')

1 2.727167557
2 2.727167557
3 2.727167557
4 2.727167557
5 2.727167557
6 2.727167557
7 2.727167557
8 2.727167557
9 2.727167557
10 2.727167557
11 2.727167557
12 2.727167557
13 2.727167557
14 2.727167557
15 2.727167557
16 2.6756217970000002
17 2.6756217970000002
18 2.6756217970000002
19 2.6756217970000002
20 2.6756217970000002
21 2.525635436
22 2.598025396
23 2.994180065
24 2.994180065
25 2.994180065
26 2.994180065
27 2.994180065
28 2.985583849
29 3.0702753510000003
30 3.1471259270000003
31 3.1471259270000003
32 3.1530800450000003
33 3.1530800450000003
34 3.1530800450000003
35 3.1530800450000003
36 3.1530800450000003
37 3.1530800450000003
38 3.1530800450000003
39 3.1530800450000003
40 3.1485387950000003
41 3.1485387950000003
42 3.1485387950000003
43 3.1485387950000003
44 3.1485387950000003
45 3.1485387950000003
46 3.1485387950000003
47 3.1485387950000003
48 3.1485387950000003
49 3.1485387950000003
50 3.1485387950000003
51 3.1485387950000003
52 3.1485387950000003
53 3.1485387950000003
54 3

In [9]:
MD_analysis(9, 'Country2', 'abs_minimax')

1 0.46740219899999996
2 0.46740219899999996
3 0.46740219899999996
4 0.46740219899999996
5 0.46740219899999996
6 0.46740219899999996
7 0.46740219899999996
8 0.46740219899999996
9 0.46740219899999996
10 0.46740219899999996
11 0.46740219899999996
12 0.46740219899999996
13 0.46740219899999996
14 0.46740219899999996
15 0.46740219899999996
16 0.46740219899999996
17 0.46740219899999996
18 0.46740219899999996
19 0.46740219899999996
20 0.46740219899999996
21 0.46740219899999996
22 0.46740219899999996
23 0.495438113
24 0.495438113
25 0.495438113
26 0.495438113
27 0.495438113
28 0.495438113
29 0.495438113
30 0.495438113
31 0.495438113
32 0.495438113
33 0.495438113
34 0.495438113
35 0.495438113
36 0.495438113
37 0.495438113
38 0.495438113
39 0.495438113
40 0.495438113
41 0.495438113
42 0.495438113
43 0.495438113
44 0.495438113
45 0.495438113
46 0.495438113
47 0.495438113
48 0.495438113
49 0.495438113
50 0.495438113
51 0.495438113
52 0.495438113
53 0.495438113
54 0.495438113
55 0.495438113
56 0.495

In [5]:
MD_analysis(9, 'Country', 'dif', 19700101, 19791231, 200108)

1 3.504997
2 3.504997
3 3.504997
4 3.6273229999999996
5 3.6273229999999996
6 3.6273229999999996
7 3.6273229999999996
8 3.6273229999999996
9 3.6273229999999996
10 3.6273229999999996
11 3.6273229999999996
12 3.6273229999999996
13 3.6273229999999996
14 3.618744
15 3.618744
16 3.618744
17 3.618744
18 3.677334
19 3.677334
20 3.677334
21 3.677334
22 3.7040919999999997
23 3.7040919999999997
24 3.7040919999999997
25 3.7040919999999997
26 3.7040919999999997
27 3.7040919999999997
28 3.7040919999999997
29 3.7040919999999997
30 3.7040919999999997
31 3.7040919999999997
32 3.7040919999999997
33 3.7238869999999995
34 3.7238869999999995
35 3.7238869999999995
36 3.727457
37 3.727457
38 3.727457
39 3.727457
40 3.727457
41 3.727457
42 3.727457
43 3.727457
44 3.7258189999999995
45 3.7258189999999995
46 3.7258189999999995
47 3.7258189999999995
48 3.674428
49 3.674428
50 3.674428
51 3.674428
52 3.674428
53 3.674428
54 3.6893279999999997
55 3.6893279999999997
56 3.6893279999999997
57 3.6893279999999997
58 3.

In [10]:
MD_analysis(11, 'S&P_sectors2')

1 4.2126
2 4.2126
3 4.2126
4 4.2126
5 4.2126
6 4.2126
7 4.2126
8 4.2126
9 4.2126
10 4.2126
11 4.2126
12 4.2126
13 4.2126
14 4.2126
15 4.2126
16 4.2126
17 4.2126
18 4.2126
19 4.2126
20 4.2126
21 4.0336
22 4.1166
23 4.431107689
24 4.431107689
25 4.431107689
26 4.444907689
27 4.444907689
28 4.428507689
29 4.445137806
30 4.445137806
31 4.445137806
32 4.445137806
33 4.445137806
34 4.445137806
35 4.445137806
36 4.445137806
37 4.427437806
38 4.427437806
39 4.418437806
40 4.389837806000001
41 4.382037806
42 4.382037806
43 4.382037806
44 4.382037806
45 4.379237806000001
46 4.381537806000001
47 4.351637805999999
48 4.351637805999999
49 4.351637805999999
50 4.351637805999999
51 4.298637806
52 4.298637806
53 4.269237806
54 4.269237806
55 4.269237806
56 4.269237806
57 4.269237806
58 4.215037806
59 4.245737805999999
60 4.245737805999999
61 4.245737805999999
62 4.245737805999999
63 4.245737805999999
64 4.177737806
65 4.177737806
66 4.177737806
67 4.177737806
68 4.124937805999999
69 4.124937805999999


In [11]:
MD_analysis(21, 'FF2')

1 7.620362999999999
2 7.620362999999999
3 7.620362999999999
4 7.620362999999999
5 7.620362999999999
6 7.620362999999999
7 7.620362999999999
8 7.620362999999999
9 7.620362999999999
10 7.620362999999999
11 7.620362999999999
12 7.620362999999999
13 7.620362999999999
14 7.620362999999999
15 7.620362999999999
16 7.620362999999999
17 7.620362999999999
18 7.620362999999999
19 7.621233999999999
20 7.621233999999999
21 6.881002
22 6.905111999999999
23 7.906872000000001
24 7.906872000000001
25 7.906872000000001
26 7.910794
27 7.91809
28 7.923074000000001
29 8.239314
30 8.239314
31 8.239314
32 8.242457000000002
33 8.242457000000002
34 8.242457000000002
35 8.242457000000002
36 8.242457000000002
37 8.242457000000002
38 8.242457000000002
39 7.668457
40 7.639234
41 7.639234
42 7.639234
43 7.494392000000001
44 7.494392000000001
45 7.494392000000001
46 7.494392000000001
47 7.494392000000001
48 7.494392000000001
49 7.494392000000001
50 7.480406
51 7.480406
52 7.480406
53 7.440661
54 7.440661
55 7.440661

In [12]:
MD_analysis(24, 'FF2')

1 8.614763
2 8.614763
3 8.614763
4 8.614763
5 8.614763
6 8.614763
7 8.614763
8 8.614763
9 8.614763
10 8.614763
11 8.614763
12 8.614763
13 8.614763
14 8.614763
15 8.614763
16 8.614763
17 8.614763
18 8.614763
19 8.615634
20 8.615634
21 7.875402
22 7.899512
23 8.901272
24 8.901272
25 8.901272
26 8.917694000000001
27 8.924990000000001
28 8.929974
29 9.339614000000001
30 9.339614000000001
31 9.339614000000001
32 9.342756999999999
33 9.342756999999999
34 9.342756999999999
35 9.342756999999999
36 9.342756999999999
37 9.342756999999999
38 9.342756999999999
39 8.671757
40 8.555834
41 8.555834
42 8.555834
43 8.333592
44 8.333592
45 8.333592
46 8.333592
47 8.333592
48 8.333592
49 8.333592
50 8.319606
51 8.268405999999999
52 8.268405999999999
53 8.228660999999999
54 8.228660999999999
55 8.228660999999999
56 8.228660999999999
57 8.228660999999999
58 8.228660999999999
59 8.367448
60 8.367448
61 8.367448
62 8.367448
63 8.367448
64 8.367448
65 8.367448
66 8.367448
67 8.367448
68 8.352848
69 8.352848
7

In [9]:
MD_analysis(16, 'IndustryIndices', 'abs_minimax', 20040101, 20061231, 201905)

1 0.47709999999999997
2 0.47709999999999997
3 0.47709999999999997
4 0.47709999999999997
5 0.3609
6 0.3609
7 0.3609
8 0.32989999999999997
9 0.32989999999999997
10 0.3256
11 0.3256
12 0.3256
13 0.3073
14 0.3073
15 0.3073
16 0.30900000000000005
17 0.30900000000000005
18 0.30900000000000005
19 0.311066242
20 0.311066242
21 0.4145
22 0.4145
23 0.5153
24 0.5153
25 0.5153
26 0.6184000000000001
27 0.6184000000000001
28 0.6184000000000001
29 0.6184000000000001
30 0.6387
31 0.6387
32 0.6387
33 0.6387
34 0.6387
35 0.6387
36 0.6387
37 0.6387
38 0.6387
39 0.6387
40 0.6387
41 0.6387
42 0.6387
43 0.6387
44 0.6387
45 0.6387
46 0.6387
47 0.6387
48 0.6387
49 0.6387
50 0.6387
51 0.6387
52 0.6387
53 0.6387
54 0.6387
55 0.6387
56 0.6387
57 0.6387
58 0.6387
59 0.5841000000000001
60 0.5841000000000001
61 0.5841000000000001
62 0.5841000000000001
63 0.5841000000000001
64 0.5841000000000001
65 0.5841000000000001
66 0.5624
67 0.5624
68 0.5624
69 0.5624
70 0.5624
71 0.5624
72 0.5624
73 0.5624
74 0.5624
75 0.5624


In [13]:
MD_analysis(16, 'IndustryIndices', 'dif', 20040101, 20061231, 201905)

1 2.629752227
2 2.58305527
3 2.58305527
4 2.588730249
5 2.301049666
6 2.301049666
7 2.304304127
8 2.26754787
9 2.3023316649999996
10 2.265631665
11 2.268374181
12 2.4148627510000003
13 2.374762751
14 2.496579917
15 2.453389515
16 2.4801338370000003
17 2.48054584
18 2.484271741
19 2.49421032
20 2.530902651
21 2.682711831
22 2.801482001
23 4.378864001
24 4.498623681
25 4.625003276999999
26 4.817703277
27 4.808049565999999
28 4.8037495660000005
29 5.498441236
30 5.736541236000001
31 5.736541236000001
32 5.736541236000001
33 5.736541236000001
34 5.736541236000001
35 5.736541236000001
36 5.737241236
37 5.742690493
38 5.742690493
39 5.742690493
40 5.742690493
41 5.736363921
42 5.736363921
43 5.736363921
44 5.736363921
45 5.736363921
46 5.736363921
47 5.736363921
48 5.736363921
49 5.736363921
50 5.736363921
51 5.736363921
52 5.736363921
53 5.749106288
54 5.753233632000001
55 5.753233632000001
56 5.753233632000001
57 5.731433632
58 5.744793614
59 4.657131919
60 4.570397891
61 4.36191551
62 4.2

In [10]:
MD_analysis(9, 'Country2', 'abs_minimax', 19970101, 20061231, 201904)

1 0.46740219899999996
2 0.46740219899999996
3 0.46740219899999996
4 0.46740219899999996
5 0.46740219899999996
6 0.46740219899999996
7 0.46740219899999996
8 0.46740219899999996
9 0.46740219899999996
10 0.46740219899999996
11 0.46740219899999996
12 0.46740219899999996
13 0.46740219899999996
14 0.46740219899999996
15 0.46740219899999996
16 0.46740219899999996
17 0.46740219899999996
18 0.46740219899999996
19 0.46740219899999996
20 0.46740219899999996
21 0.46740219899999996
22 0.46740219899999996
23 0.495438113
24 0.495438113
25 0.495438113
26 0.495438113
27 0.495438113
28 0.495438113
29 0.495438113
30 0.495438113
31 0.495438113
32 0.495438113
33 0.495438113
34 0.495438113
35 0.495438113
36 0.495438113
37 0.495438113
38 0.495438113
39 0.495438113
40 0.495438113
41 0.495438113
42 0.495438113
43 0.495438113
44 0.495438113
45 0.495438113
46 0.495438113
47 0.495438113
48 0.495438113
49 0.495438113
50 0.495438113
51 0.495438113
52 0.495438113
53 0.495438113
54 0.495438113
55 0.495438113
56 0.495

In [12]:
MD_analysis(3, 'MKT2', 'abs_minimax', 19970101, 20061231, 201904)

1 0.3801
2 0.3801
3 0.3801
4 0.3801
5 0.3801
6 0.3801
7 0.3801
8 0.3801
9 0.3801
10 0.3801
11 0.3801
12 0.3801
13 0.3801
14 0.3801
15 0.3801
16 0.3801
17 0.3801
18 0.3801
19 0.3801
20 0.3801
21 0.3801
22 0.3801
23 0.3842
24 0.3842
25 0.3842
26 0.3842
27 0.3842
28 0.3842
29 0.3842
30 0.3842
31 0.3842
32 0.3842
33 0.3842
34 0.3842
35 0.3842
36 0.3842
37 0.3842
38 0.3842
39 0.3003
40 0.3003
41 0.3003
42 0.3003
43 0.29810000000000003
44 0.29810000000000003
45 0.29810000000000003
46 0.29810000000000003
47 0.29810000000000003
48 0.29810000000000003
49 0.29810000000000003
50 0.29810000000000003
51 0.2741
52 0.2741
53 0.2741
54 0.2741
55 0.2741
56 0.2741
57 0.2741
58 0.2741
59 0.2858
60 0.2858
61 0.2858
62 0.2858
63 0.2858
64 0.2858
65 0.2858
66 0.2858
67 0.2858
68 0.2858
69 0.2858
70 0.2858
71 0.2858
72 0.2858
73 0.2858
74 0.2858
75 0.2858
76 0.2858
77 0.2858
78 0.2858
79 0.2858
80 0.2858
81 0.2858
82 0.2858
83 0.2858
84 0.2858
85 0.2858
86 0.2858
87 0.2858
88 0.2858
89 0.2858
90 0.2858
91 0.

In [14]:
MD_analysis(21, 'FF2', 'abs_minimax', 19970101, 20061231, 201904)

1 0.664107
2 0.664107
3 0.664107
4 0.664107
5 0.664107
6 0.664107
7 0.664107
8 0.664107
9 0.664107
10 0.664107
11 0.664107
12 0.664107
13 0.664107
14 0.664107
15 0.664107
16 0.664107
17 0.664107
18 0.664107
19 0.664107
20 0.664107
21 0.628641
22 0.628641
23 0.680678
24 0.680678
25 0.680678
26 0.680678
27 0.680678
28 0.680678
29 0.680678
30 0.680678
31 0.680678
32 0.680678
33 0.680678
34 0.680678
35 0.680678
36 0.680678
37 0.680678
38 0.680678
39 0.540223
40 0.540223
41 0.540223
42 0.540223
43 0.46165900000000004
44 0.46165900000000004
45 0.46165900000000004
46 0.46165900000000004
47 0.46165900000000004
48 0.46165900000000004
49 0.46165900000000004
50 0.46165900000000004
51 0.46165900000000004
52 0.46165900000000004
53 0.46165900000000004
54 0.46165900000000004
55 0.46165900000000004
56 0.46165900000000004
57 0.46165900000000004
58 0.46165900000000004
59 0.46165900000000004
60 0.46165900000000004
61 0.46165900000000004
62 0.46165900000000004
63 0.46165900000000004
64 0.46165900000000004

In [15]:
MD_analysis(24, 'FF2', 'abs_minimax', 19970101, 20061231, 201904)

1 0.664107
2 0.664107
3 0.664107
4 0.664107
5 0.664107
6 0.664107
7 0.664107
8 0.664107
9 0.664107
10 0.664107
11 0.664107
12 0.664107
13 0.664107
14 0.664107
15 0.664107
16 0.664107
17 0.664107
18 0.664107
19 0.664107
20 0.664107
21 0.661016
22 0.661016
23 0.680678
24 0.680678
25 0.680678
26 0.680678
27 0.680678
28 0.680678
29 0.754416
30 0.754416
31 0.754416
32 0.754416
33 0.754416
34 0.754416
35 0.754416
36 0.754416
37 0.754416
38 0.754416
39 0.613961
40 0.613961
41 0.613961
42 0.613961
43 0.535397
44 0.535397
45 0.535397
46 0.535397
47 0.535397
48 0.535397
49 0.535397
50 0.535397
51 0.535397
52 0.535397
53 0.535397
54 0.535397
55 0.535397
56 0.535397
57 0.535397
58 0.535397
59 0.535397
60 0.535397
61 0.535397
62 0.535397
63 0.535397
64 0.535397
65 0.535397
66 0.535397
67 0.535397
68 0.535397
69 0.535397
70 0.535397
71 0.535397
72 0.535397
73 0.535397
74 0.535397
75 0.535397
76 0.535397
77 0.535397
78 0.535397
79 0.535397
80 0.535397
81 0.535397
82 0.535397
83 0.535397
84 0.535397
8

In [17]:
MD_analysis(11, 'S&P_sectors2', 'abs_minimax', 19970101, 20061231, 201904)

1 0.6930000000000001
2 0.6930000000000001
3 0.6930000000000001
4 0.6930000000000001
5 0.6930000000000001
6 0.6930000000000001
7 0.6930000000000001
8 0.6930000000000001
9 0.6930000000000001
10 0.6930000000000001
11 0.6930000000000001
12 0.6930000000000001
13 0.6930000000000001
14 0.6930000000000001
15 0.6930000000000001
16 0.6930000000000001
17 0.6930000000000001
18 0.6930000000000001
19 0.6930000000000001
20 0.6930000000000001
21 0.6930000000000001
22 0.6930000000000001
23 0.6930000000000001
24 0.6930000000000001
25 0.6930000000000001
26 0.6930000000000001
27 0.6930000000000001
28 0.6930000000000001
29 0.6930000000000001
30 0.6930000000000001
31 0.6930000000000001
32 0.6930000000000001
33 0.6930000000000001
34 0.6930000000000001
35 0.6930000000000001
36 0.6930000000000001
37 0.6930000000000001
38 0.6930000000000001
39 0.6930000000000001
40 0.6930000000000001
41 0.6930000000000001
42 0.6930000000000001
43 0.6930000000000001
44 0.6930000000000001
45 0.6930000000000001
46 0.69300000000000